# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

In [37]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

### Hit rate@k

In [3]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [4]:
hit_rate_at_k(recommended_list, bought_list, k=3)

1

### Money Precision@k

In [7]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    prices_recommended = np.array(prices_recommended[:k])
    flags = np.isin(recommended_list[:k], bought_list)
    
    precision = (flags * prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [8]:
prices_recommended = [400, 60, 40, 40 , 90]

In [9]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.6984126984126984

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [91]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = (flags * prices_bought).sum() / prices_bought.sum()
    
    return recall

In [93]:
recall_at_k(recommended_list, bought_list, k=3)

0.25

In [12]:
prices_bought = [40, 60, 400, 40]

In [13]:
money_recall_at_k(recommended_list, bought_list, prices_bought, k=3)

0.7407407407407407

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [89]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    rank = np.nonzero(flags)[0][0]
    
    return 1/(rank + 1)

In [90]:
mrr_at_k(recommended_list, bought_list, k=3)

1.0

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [87]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    discount = np.append([1, 2], np.log2(np.arange(3, k+1)))
    
    IDCG = (np.array([True] * k) / discount).sum()
    DCG = (flags / discount).sum()
    
    return DCG / IDCG

In [94]:
ndcg_at_k(recommended_list, bought_list, k=3)

0.46927872602275644